In [106]:
import os
import sys
import time
import gc


# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
from scipy import stats

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

#Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# Machine Learning models
import lightgbm as lgb
from sklearn import linear_model


In [107]:
data_raw = pd.read_csv('../PUBG/train_V2.csv')
data_val  = pd.read_csv('../PUBG/test_V2.csv')
data1 = data_raw.copy(deep = True)
#data1=pd.read_csv('../PUBG/train_V2.csv')
#data_cleaner = [data1, data_val]

display(data_raw.info(verbose= True))
display(data_raw.describe(include='all'))
data_raw.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 983.9+ MB


None

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,4446966,4446966,4446966,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,...,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446965e+06
unique,4446966,2026745,47965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,4858c47c28c30c,14d6b54cdec6bc,8611c2a3adb089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,74,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,2.338149e-01,1.106908e+00,1.307171e+02,6.578755e-01,2.268196e-01,1.370147e+00,4.759935e+01,...,1.646590e-01,6.061157e+02,3.496091e-03,4.509322e+00,2.386841e-02,7.918208e-03,1.154218e+03,3.660488e+00,6.064601e+02,4.728216e-01
std,NaN,NaN,NaN,5.885731e-01,1.715794e+00,1.707806e+02,1.145743e+00,6.021553e-01,2.679982e+00,2.746294e+01,...,4.721671e-01,1.498344e+03,7.337297e-02,3.050220e+01,1.673935e-01,9.261157e-02,1.183497e+03,2.456544e+00,7.397004e+02,3.074050e-01
min,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.551000e+02,2.000000e+00,0.000000e+00,2.000000e-01
50%,NaN,NaN,NaN,0.000000e+00,0.000000e+00,8.424000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.856000e+02,3.000000e+00,0.000000e+00,4.583000e-01
75%,NaN,NaN,NaN,0.000000e+00,2.000000e+00,1.860000e+02,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,...,0.000000e+00,1.909750e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.976000e+03,5.000000e+00,1.495000e+03,7.407000e-01


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
4139798,028107c6c25d98,2fdb3ab15caacd,c16b69e4aef259,0,1,0.00,0,0,0,54,...,0,0.0,0,0.0,0,0,1022.0,4,0,0.5769
1939294,2cea36887fb02f,368f4e98889462,37eb0a39e6649d,1,1,166.70,0,0,0,66,...,0,0.0,0,0.0,0,0,350.5,5,1592,0.3913
3712176,30bc2d2b0c35b2,3973ca2f87b649,633905b884fcbc,0,1,83.00,0,0,0,28,...,0,0.0,0,0.0,0,0,3601.0,2,0,0.8800
2320239,f0bc718f909f0a,5d1039a968514a,9fddbc9d3ec6ed,3,1,363.80,4,0,0,4,...,1,330.7,0,0.0,0,0,2870.0,8,1601,0.9259
2458406,d893cc15399cf0,bed43f38e48919,02c119df7551bd,0,0,0.00,0,0,0,47,...,0,0.0,0,0.0,0,0,3204.0,5,0,0.6538
929028,871f432fede3c3,3c6e19cb7c0563,bf55848e6397a1,0,0,221.70,2,0,2,24,...,1,0.0,0,0.0,0,0,1946.0,3,1561,0.7500
29042,625591b7a255f2,6d2e7d8a6691a5,ede9775dae4c67,0,0,0.00,0,0,0,79,...,0,0.0,0,0.0,0,0,111.9,4,0,0.1613
2836512,b7cbdc94604ae6,c341c2a73411e1,f155bb2e598c4f,3,4,493.40,1,1,1,15,...,0,0.0,0,0.0,0,0,2488.0,3,1448,0.7407
3453702,af87f22190bdc8,9accc983b05a33,a4c1b43cf5eccf,0,0,73.34,0,0,0,58,...,1,0.0,0,0.0,0,0,2092.0,3,1543,0.6429
136649,306fe381396eaf,6d14b10203dd54,9394a933cb882f,0,3,73.53,0,0,2,49,...,1,0.0,0,0.0,0,0,2004.0,4,1500,0.7778


In [108]:
def reduce_size(merged_df):
    print('      Starting size is %d Mb'%(sys.getsizeof(merged_df)/1024/1024))
    print('      Columns: %d'%(merged_df.shape[1]))
    feats = merged_df.columns[merged_df.dtypes == 'float64']
    for feat in feats:
        merged_df[feat] = merged_df[feat].astype('float32')

    feats = merged_df.columns[merged_df.dtypes == 'int16']
    for feat in feats:
        mm = np.abs(merged_df[feat]).max()
        if mm < 126:
            merged_df[feat] = merged_df[feat].astype('int8')

    feats = merged_df.columns[merged_df.dtypes == 'int32']
    for feat in feats:
        mm = np.abs(merged_df[feat]).max()
        if mm < 126:
            merged_df[feat] = merged_df[feat].astype('int8')
        elif mm < 30000:
            merged_df[feat] = merged_df[feat].astype('int16')

    feats = merged_df.columns[merged_df.dtypes == 'int64']
    for feat in feats:
        mm = np.abs(merged_df[feat]).max()
        if mm < 126:
            merged_df[feat] = merged_df[feat].astype('int8')
        elif mm < 30000:
            merged_df[feat] = merged_df[feat].astype('int16')
        elif mm < 2000000000:
            merged_df[feat] = merged_df[feat].astype('int32')
    print('      Ending size is %d Mb'%(sys.getsizeof(merged_df)/1024/1024))
    return merged_df

In [109]:
#reducing memory occupation
data1_red=reduce_size(data1)
data_val_red=reduce_size(data_val)

      Starting size is 2024 Mb
      Columns: 29
      Ending size is 1379 Mb
      Starting size is 865 Mb
      Columns: 28
      Ending size is 592 Mb


In [110]:
print('Train columns with null values:\n', data1_red.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data1_red.isnull().sum())
print("-"*10)

data1_red.describe(include = 'all')

Train columns with null values:
 Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64
----------
Test/Validation columns with null values:
 Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill       

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
count,4446966,4446966,4446966,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,...,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446966e+06,4.446965e+06
unique,4446966,2026745,47965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,4858c47c28c30c,14d6b54cdec6bc,8611c2a3adb089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,74,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,2.338149e-01,1.106908e+00,1.306331e+02,6.578755e-01,2.268196e-01,1.370147e+00,4.759935e+01,...,1.646590e-01,6.060923e+02,3.496091e-03,4.509240e+00,2.386841e-02,7.918208e-03,1.148517e+03,3.660488e+00,6.064601e+02,4.728141e-01
std,NaN,NaN,NaN,5.885731e-01,1.715794e+00,1.698869e+02,1.145743e+00,6.021553e-01,2.679982e+00,2.746294e+01,...,4.721671e-01,1.496470e+03,7.337297e-02,3.023784e+01,1.673935e-01,9.261157e-02,1.180553e+03,2.456544e+00,7.397004e+02,3.068041e-01
min,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.400000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.551000e+02,2.000000e+00,0.000000e+00,2.000000e-01
50%,NaN,NaN,NaN,0.000000e+00,0.000000e+00,8.424000e+01,0.000000e+00,0.000000e+00,0.000000e+00,4.700000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.856000e+02,3.000000e+00,0.000000e+00,4.583000e-01
75%,NaN,NaN,NaN,0.000000e+00,2.000000e+00,1.860000e+02,1.000000e+00,0.000000e+00,2.000000e+00,7.100000e+01,...,0.000000e+00,1.909750e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.976000e+03,5.000000e+00,1.495000e+03,7.407000e-01


In [111]:
#Cleaning Match type other than standard types
print('Match types BEFORE filtering:\n', data1_red.matchType.unique())
standard_matches=['solo', 'duo', 'squad', 'solo-fpp', 'duo-fpp', 'squad-fpp']
#mask = (data1_red['matchType']).isin(standard_matches)
data1_red.loc[~data1_red['matchType'].isin(standard_matches), 'matchType']= 'other'
data_val_red.loc[~data_val_red['matchType'].isin(standard_matches), 'matchType']= 'other'
print('Match types AFTER filtering:\n',data1_red.matchType.unique())


Match types BEFORE filtering:
 ['squad-fpp' 'duo' 'solo-fpp' 'squad' 'duo-fpp' 'solo' 'normal-squad-fpp'
 'crashfpp' 'flaretpp' 'normal-solo-fpp' 'flarefpp' 'normal-duo-fpp'
 'normal-duo' 'normal-squad' 'crashtpp' 'normal-solo']
Match types AFTER filtering:
 ['squad-fpp' 'duo' 'solo-fpp' 'squad' 'duo-fpp' 'solo' 'other']


In [112]:
#preparing data values as described into challenge

data_cleaner=[data1_red, data_val_red]
for dataset in data_cleaner:
    #killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
    #dataset[(dataset['rankPoints']!=-1) & (dataset['killPoints']==0)]['killPoints']=None
    dataset.loc[(dataset['rankPoints']!=-1) & (dataset['killPoints']==0),'killPoints']=np.NaN
    #winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
    dataset.loc[(dataset['rankPoints']!=-1) & (dataset['winPoints']==0),'winPoints']=np.NaN
    #rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
    dataset.loc[(dataset['rankPoints']==-1),'rankPoints']=np.NaN
    
    

In [113]:
#Filling missing values
colsToMeanByMatch=['rankPoints', 'killPoints']
for dataset in data_cleaner:
    for colonna in colsToMeanByMatch:
        dataset.loc[:,colonna].fillna(dataset[colonna].mean(), inplace=True)


In [114]:
#creating some new useful features
#orig_col=data1_red.columns.values
#print(orig_col)
#to_drop_cols=['Id', 'groupId', 'matchId']
for dataset in data_cleaner:
    dataset.loc[:,'hsRatio']=dataset['headshotKills'].div(dataset['kills'].where(dataset['headshotKills']!=0, other=np.inf))
    dataset.loc[:,'skillPoints']=dataset['headshotKills']+dataset['roadKills']+dataset['longestKill']+dataset['vehicleDestroys']+dataset['roadKills']-dataset['teamKills']
    dataset.loc[:,'totalDistance']=dataset['walkDistance']+dataset['swimDistance']+dataset['rideDistance']
engFeatures=['headshotKills', 'skillPoints', 'totalDistance']

In [115]:
#Label Encoding for match type
one_hot=pd.DataFrame()
for dataset in data_cleaner:
    one_hot = pd.get_dummies(dataset['matchType'])
    # Drop column as it is now encoded
    #dataset.drop(columns='matchType',axis = 1, inplace=True)
    # Join the encoded df
    dataset.loc[:,:][cols]=one_hot.copy(deep=True)
one_hot_cols=one_hot.columns.values
print(one_hot_cols)

['duo' 'duo-fpp' 'other' 'solo' 'solo-fpp' 'squad' 'squad-fpp']


In [116]:
#distinguishing columns by data type
numeric_columns=['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration', 'maxPlace', 'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints', 'totalDistance', 'skillPoints']
categorical_columns=['matchType'] # maybe i will include n. of team members or boolean is team
todrop_columns=['rankPoints', 'Id', 'matchId', 'groupId', 'matchType', 'winPoints']
Y_cols='winPlacePerc'

print(X_cols)

['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration', 'maxPlace', 'numGroups', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'hsRatio', 'skillPoints', 'totalDistance', 'duo', 'duo-fpp', 'other', 'solo', 'solo-fpp', 'squad', 'squad-fpp']


In [117]:
#normalizing numeric features
scaler = MinMaxScaler() 
for dataset in data_cleaner:
    scaled_values = scaler.fit_transform(dataset[numeric_columns]) 
    dataset.loc[:,numeric_columns] = scaled_values
    dataset=reduce_size(dataset)

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int8, int16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


      Starting size is 2138 Mb
      Columns: 39
      Ending size is 1680 Mb


D:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int8, int16, float32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


      Starting size is 922 Mb
      Columns: 38
      Ending size is 723 Mb


In [118]:
#dropping useless columns
#for dataset in data_cleaner:
#    dataset.drop(columns=todrop_columns, axis=1, inplace=True)
#    print(dataset.info())


In [119]:
#print(data1_red.isnull().values.any())
#print(data1_red.isnull().any)
#nan_rows = data1_red[data1_red.isnull()]
#display(nan_rows)
#senzanull=data1_red.dropna(axis=0)
#print(senzanull.info())
data1_red.dropna(subset=['winPlacePerc'], axis=0, inplace=True)
data1_red.isnull().sum()

Id                       0
groupId                  0
matchId                  0
assists                  0
boosts                   0
damageDealt              0
DBNOs                    0
headshotKills            0
heals                    0
killPlace                0
killPoints               0
kills                    0
killStreaks              0
longestKill              0
matchDuration            0
matchType                0
maxPlace                 0
numGroups                0
rankPoints               0
revives                  0
rideDistance             0
roadKills                0
swimDistance             0
teamKills                0
vehicleDestroys          0
walkDistance             0
weaponsAcquired          0
winPoints          2655646
winPlacePerc             0
hsRatio                  0
skillPoints              0
totalDistance            0
duo                      0
duo-fpp                  0
other                    0
solo                     0
solo-fpp                 0
s

In [122]:
Y_cols='winPlacePerc'
X_cols=data1_red.columns.tolist()
X_cols.remove(Y_cols)
for elem in todrop_columns:
    X_cols.remove(elem)
Y_train=data1_red[Y_cols]
X_train=data1_red[X_cols]
SGDReg = linear_model.SGDRegressor(max_iter=1000, tol=1e-3, )
SGDReg.fit(X_train,Y_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=1000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1,
       verbose=0, warm_start=False)

In [127]:
data_val_red['winPlacePerc']=SGDReg.predict(data_val_red[X_cols])
print(data_val_red['winPlacePerc'].describe())
print(data_val_red.info())

data_val_red[['Id', 'winPlacePerc']].to_csv("../submit.csv", index=False)

count    1.934174e+06
mean     5.352319e-01
std      3.432579e-01
min     -1.539896e+00
25%      2.541376e-01
50%      4.704075e-01
75%      8.076028e-01
max      2.937249e+00
Name: winPlacePerc, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934174 entries, 0 to 1934173
Data columns (total 39 columns):
Id                 object
groupId            object
matchId            object
assists            float32
boosts             float32
damageDealt        float32
DBNOs              float32
headshotKills      float32
heals              float32
killPlace          float32
killPoints         float32
kills              float32
killStreaks        float32
longestKill        float32
matchDuration      float32
matchType          object
maxPlace           float32
numGroups          float32
rankPoints         float32
revives            float32
rideDistance       float32
roadKills          float32
swimDistance       float32
teamKills          float32
vehicleDestroys    float32
walkD